<a href="https://colab.research.google.com/github/mamagoudou/QNN-with-dithering/blob/main/notebooks/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import math

import csv
import time
from tqdm.notebook import tqdm


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

PATH_Models = '/content/drive/MyDrive/Memory/Models/ResNet/'
PATH_Measures = '/content/drive/MyDrive/Memory/Measures/ResNet/'

# Model definition

## BasicBlock module definition
Inspired: [Ksuryate](https://github.com/Ksuryateja/pytorch-cifar10/blob/master/models/resnet.py)

Paper: [Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf)

In [2]:
class BasicBlock(nn.Module):

  expansion = 1
  def __init__(self, in_planes, planes, stride=1):

    super(BasicBlock, self).__init__()
    self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, 
                            padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(planes)
    self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, 
                            padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(planes)

    self.shortcut = nn.Sequential()
    if stride != 1 or in_planes != self.expansion*planes:
        self.shortcut = nn.Sequential(
            nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, 
                      stride=stride, bias=False),
            nn.BatchNorm2d(self.expansion*planes)
        )

  def forward(self, x):
    y = F.relu(self.bn1(self.conv1(x)))
    y = self.bn2(self.conv2(y))
    y += self.shortcut(x)
    y = F.relu(y)

    return y

## Bottleneck module

In [3]:
class Bottleneck(nn.Module):

  expansion = 4
  def __init__(self, in_planes, planes, stride=1):
    super(Bottleneck, self).__init__()
    self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
    self.bn1 = nn.BatchNorm2d(planes)
    self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, 
                            padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(planes)
    self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, 
                            bias=False)
    self.bn3 = nn.BatchNorm2d(self.expansion*planes)

    self.shortcut = nn.Sequential()
    if stride != 1 or in_planes != self.expansion*planes:
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, 
                    stride=stride, bias=False),
          nn.BatchNorm2d(self.expansion*planes)
      )

  def forward(self, x):
    y = F.relu(self.bn1(self.conv1(x)))
    y = F.relu(self.bn2(self.conv2(y)))
    y = self.bn3(self.conv3(y))
    y += self.shortcut(x)
    y = F.relu(y)
    return y

## ResNet module

In [4]:
class ResNet(nn.Module):

  def __init__(self, block, num_blocks):

    super(ResNet, self).__init__()
    self.in_planes = 64

    self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(64)
    self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
    self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
    self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
    self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
    self.linear = nn.Linear(512*block.expansion, 10)

    # Initialize weights
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        m.weight.data.normal_(0, math.sqrt(2. / n))
        if m.bias:
          m.bias.data.zero_()

  def _make_layer(self, block, planes, num_blocks, stride):
    strides = [stride] + [1]*(num_blocks-1)
    layers = []
    for stride in strides:
      layers.append(block(self.in_planes, planes, stride))
      self.in_planes = planes * block.expansion

    return nn.Sequential(*layers)

  def forward(self, x):
    x = F.relu(self.bn1(self.conv1(x)))
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = F.avg_pool2d(x, 4)
    x = x.view(x.size(0), -1)
    x = self.linear(x)

    return x

In [5]:
def ResNet18():
  return ResNet(BasicBlock, [2,2,2,2])

def ResNet34():
  return ResNet(BasicBlock, [3,4,6,3])

def ResNet50():
  return ResNet(Bottleneck, [3,4,6,3])

def ResNet101():
  return ResNet(Bottleneck, [3,4,23,3])

def ResNet152():
  return ResNet(Bottleneck, [3,8,36,3])

## Build network

In [6]:
# NAME_DD_MM_TEST
PATH_Name = 'ResNet18_15_02_TEST'

network = ResNet18()

epoch = 0
network.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

# Dataset loading, processing and loaders

In [7]:
BATCH_SIZE = 128
NUM_WORKERS = 4

train_transform = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), 
                                                           (0.2023, 0.1994, 0.2010))])

test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.4914, 0.4822, 0.4465), 
                                                          (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transform)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=NUM_WORKERS)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=NUM_WORKERS)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# Training & validation

## Optimizer and scheduler definition

In [8]:
LEARNING_RATE = 0.1
MOMENTUM = 0.9
NEPOCHS  = 50
TOTAL_STEPS = math.ceil(len(trainloader.dataset)/BATCH_SIZE)*NEPOCHS
CRITERION = "CrossEntropyLoss"
OPTIMIZER = "SGD"
SCHEDULER = "OneCycleLR"

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(network.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
lr_scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.1,
                                             total_steps=TOTAL_STEPS,
                                             anneal_strategy='linear')

## Model training, testing and stats

In [ ]:
TrainLoss = []
TrainAcc = []
Traintime = []
TestLoss = []
TestAcc = []

for epoch in tqdm(range(epoch, NEPOCHS), position=0, desc="Epoch"):

  print("Epoch: %d" %(epoch))
  # TRAINING
  network.train()
  start_time = time.time()
  train_loss = 0
  correct = 0
  total = 0
  for i, data in tqdm(enumerate(trainloader, 0), position=1, desc="Training", 
                      total=len(trainloader.dataset)/BATCH_SIZE, leave=False):
    
    inputs, labels = data[0].to(device), data[1].to(device)
    optimizer.zero_grad()
    outputs = network(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    train_loss += loss.item()
    _, predicted = outputs.max(1)
    total += labels.size(0)
    correct += predicted.eq(labels).sum().item()
    end_time = time.time()

  lr_scheduler.step()
  TrainLoss.append(train_loss/(i+1))
  TrainAcc.append(100.*correct/total)
  Traintime.append(end_time-start_time)
  print('TrainLoss: %.3f | TrainAcc: %.3f%% (%d/%d) | Time Elapsed %.3f sec' 
        % (TrainLoss[-1], TrainAcc[-1], correct, total, Traintime[-1]))
  
  # TESTING
  network.eval()
  test_loss = 0
  correct = 0
  total = 0
  with torch.no_grad():
    for i, data in tqdm(enumerate(testloader, 0), position=2, desc="Testing", 
                        total=len(testloader.dataset)/BATCH_SIZE, leave=False):
      inputs, labels = data[0].to(device), data[1].to(device)
      outputs = network(inputs)
      loss = criterion(outputs, labels)

      test_loss += loss.item()
      _, predicted = outputs.max(1)
      total += labels.size(0)
      correct += predicted.eq(labels).sum().item()

    TestLoss.append(test_loss/(i+1))
    TestAcc.append(100.*correct/total)
    print('TestLoss: %.3f | TestAcc: %.3f%% (%d/%d)' 
          % (TestLoss[-1], TestAcc[-1], correct, total))
    print('-' * 75)
  # SAVE MODEL IF BEST
  if TestAcc[-1] == max(TestAcc):
    torch.save({
      'optimizer': optimizer.state_dict(),
      'scheduler': lr_scheduler.state_dict(),
      'network': network.state_dict(),
      'epoch': epoch
    }, PATH_Models + PATH_Name + '.pth')


# WRITE INFOS & STATS IN CSV
stats = {"TrainLoss": TrainLoss, "TrainAcc": TrainAcc, "Traintime": Traintime,
         "TestLoss": TestLoss, "TestAcc": TestAcc}

with open(PATH_Measures + PATH_Name + ".csv", "w") as f:
  writer = csv.writer(f)
  writer.writerow(stats.keys())
  writer.writerows(zip(*stats.values()))

infos = {
  "PATH_Name":PATH_Name,
  "BATCH_SIZE":BATCH_SIZE,
  "NUM_WORKERS":NUM_WORKERS,
  "LEARNING_RATE":LEARNING_RATE,
  "MOMENTUM":MOMENTUM,
  "NEPOCHS":NEPOCHS,
  "TOTAL_STEPS":TOTAL_STEPS,
  "CRITERION":CRITERION,
  "OPTIMIZER":OPTIMIZER,
  "SCHEDULER":SCHEDULER,
  "optimizer":optimizer.state_dict(),
  "lr_scheduler":lr_scheduler.state_dict(),
  "epoch":epoch,
}

with open(PATH_Measures + PATH_Name + "_infos.csv", "w") as f:
  writer = csv.DictWriter(f, fieldnames=infos.keys())
  writer.writeheader()
  writer.writerow(infos)

Epoch: 0


TrainLoss: 1.621 | TrainAcc: 39.696% (19848/50000) | Time Elapsed 65.083 sec


TestLoss: 1.290 | TestAcc: 52.900% (5290/10000)
---------------------------------------------------------------------------
Epoch: 1


TrainLoss: 1.152 | TrainAcc: 58.452% (29226/50000) | Time Elapsed 64.275 sec


TestLoss: 1.187 | TestAcc: 59.920% (5992/10000)
---------------------------------------------------------------------------
Epoch: 2


## Backup for savings

In [ ]:
# WRITE INFOS & STATS IN CSV
stats = {"TrainLoss": TrainLoss, "TrainAcc": TrainAcc, "Traintime": Traintime,
         "TestLoss": TestLoss, "TestAcc": TestAcc}

with open(PATH_Measures + PATH_Name + ".csv", "w") as f:
  writer = csv.writer(f)
  writer.writerow(stats.keys())
  writer.writerows(zip(*stats.values()))

infos = {
  "PATH_Name":PATH_Name,
  "BATCH_SIZE":BATCH_SIZE,
  "NUM_WORKERS":NUM_WORKERS,
  "LEARNING_RATE":LEARNING_RATE,
  "MOMENTUM":MOMENTUM,
  "NEPOCHS":NEPOCHS,
  "TOTAL_STEPS":TOTAL_STEPS,
  "CRITERION":CRITERION,
  "OPTIMIZER":OPTIMIZER,
  "SCHEDULER":SCHEDULER,
  "optimizer":optimizer.state_dict(),
  "lr_scheduler":lr_scheduler.state_dict(),
  "epoch":epoch,
}

with open(PATH_Measures + PATH_Name + "_infos.csv", "w") as f:
  writer = csv.DictWriter(f, fieldnames=infos.keys())
  writer.writeheader()
  writer.writerow(infos)

# Drafts